In [2]:
# update your password
# from mlutils import storage
# storage.update_secret("password_for_sharepoint_bq_transfer", "YOUR_PASS_WORD")

In [1]:
from sharepoint_pricingengine import get_data, clean_data, create_log, send_email, to_bq, main, create_excel_file

In [2]:
df = get_data()

In [3]:
# manually add a duplicated row
df.loc[len(df.index)] = ['1021', 'Advance Auto Parts', '104081', 'Small Parcel', '2022-07-26', 'Current', 'Manual']

In [4]:
# manually add a missing row
df.loc[len(df.index)] = ['1021', 'Advance Auto Parts', '104081', 'Small Parcel', None, 'Current', 'Manual']

In [5]:
missing, duplicated = clean_data(df)

In [6]:
missing

,Client_ID,Client_Name,Pickup_Point_ID,Type_Of_Delivery,Start_Date,End_Date,Source
16537,1021,Advance Auto Parts,104081,Small Parcel,None,2262-04-11,Manual


In [7]:
duplicated

,Client_ID,Client_Name,Pickup_Point_ID,Type_Of_Delivery,Start_Date,End_Date,Source
16535,1021,Advance Auto Parts,104081,Small Parcel,2022-07-26,2262-04-11,Manual
16536,1021,Advance Auto Parts,104081,Small Parcel,2022-07-26,2262-04-11,Manual


In [8]:
# test create_excel_file function
create_excel_file({'Missing': missing, 'Duplicated': duplicated})

In [11]:
bq_res = to_bq(df)
is_success, log_body = bq_res
# send log based on upload to BigQuery result
if is_success:
    attachment = create_excel_file({'Missing': missing, 'Duplicated': duplicated})
    if attachment is not None:
        log_body += log_body_with_attachment
        msg = create_log(log_body, is_failed=False, attachment=attachment)
        send_email(msg)
    else:
        msg = create_log(log_body, is_failed=True)
        send_email(msg)

In [9]:
# test fail email notification
log_body = "TEST on FAIL"
msg = create_log(log_body, is_failed=True)
send_email(msg)